<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/HateEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 8.9MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 2.9MB 51.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8403cd4e2d417d5bfcbd8dbcf559f49f2f2c1e2917bee8b014be30c3678e15ec
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
cd /content/drive/My Drive/HateSpeechDetection/Dataset/Hate/No-Prc/

/content/drive/My Drive/HateSpeechDetection/Dataset/Hate/No-Prc


In [ ]:
!ls

HateEval  OffenseEval  single_runs


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('test.csv')
len(df)

3000

In [ ]:
res = df[df["label"]==1]
len(res)

1260

In [ ]:
# def df2dl(df):
#     df = df.dropna()
#     df.text = df.text.apply(lambda x: "[CLS] " + x + ' [SEP]')
#     tokenized_texts = df.text.apply(lambda x: tokenizer.tokenize(' '.join(x.split()[:MAX_LEN])))
#     token_ids = tokenized_texts.apply(tokenizer.convert_tokens_to_ids)
#     token_ids_matrix = np.array(token_ids.apply(lambda x: x[:MAX_LEN] + [0] * max(0, MAX_LEN - len(x))).tolist()).astype('int64')
#     attention_matrix = (token_ids_matrix != 0).astype('float')
#     dataset = TensorDataset(torch.tensor(token_ids_matrix), torch.tensor(attention_matrix), torch.tensor(np.array(df.label)))
#     data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
#     return data_loader

In [ ]:
# import pandas as pd
# df = pd.read_csv('train.csv')
# df.dropna(inplace=True)
# label_counts = df.label.value_counts(normalize=True)
# train_data_loader = df2dl(df)

In [ ]:
# from transformers import AutoConfig, AlbertModel
# albert_xxlarge_configuration = AlbertConfig()
# # Initializing an ALBERT-base style configuration

In [ ]:
# albert_base_configuration = AutoConfig(
#       num_attention_heads=12,
#       intermediate_size=3072,
#   )

#  # Initializing a model from the ALBERT-base style configuration
# model = AlbertModel(albert_base_configuration)

In [ ]:
# configuration = model.config

In [ ]:
# configuration

In [ ]:
            if(dataset_names[d_index][:4]=='Offe'):
              offe_f_one.append(metrics_test[2])
              offe_accuracy.append(metrics_test[3])
            elif (dataset_names[d_index][:4]=='Hate'):
              hate_f_one.append(metrics_test[2])
              hate_accuracy.append(metrics_test[3])
            else:
              sent_f_one.append(metrics_test[2])
              sent_accuracy.append(metrics_test[3])

            if(len(hate_f_one)>20 and len(hate_f_one)>20):
              # print("mohammad",max(hate_f_one),max(offe_f_one))
              if((hate_f_one[-1]<hate_f_one[-21]) and (offe_f_one[-1]<offe_f_one[-21])):
                offe_index = offe_f_one.index(max(offe_f_one))
                hate_index = hate_f_one.index(max(hate_f_one))
                print("\nearly stopping at {}\n \
                        Offensive max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Hate max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}"\
                        .format(i_epoch ,
                                max(offe_f_one) , offe_accuracy[offe_index], offe_index+1
                                max(hate_f_one) , hate_accuracy[hate_index], hate_index+1))


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.7MB/s 


In [ ]:
import os
import random
import datetime, time
from shutil import rmtree
import transformers
from torch.utils import tensorboard
import torch
from sklearn.metrics import confusion_matrix
from torch import nn
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from torch.utils.tensorboard.summary import hparams
import transformers
import tqdm
from torch.nn import functional as F
from tqdm import tqdm
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
 
# %%
import argparse
 
parser = argparse.ArgumentParser()
 
# parser.add_argument('--dataset', default='OffenseEval')
# parser.add_argument('--max_len', default=60, type=int)
# parser.add_argument('--epochs', default=5, type=int)
# parser.add_argument('--num_classes', default=2, type=int)
# parser.add_argument('--batch', default=64, type=int)
#
 
# parser.add_argument('--dataset', default='Hate/Conditions/4-Remove_Mention_Sign/HateEval')
parser.add_argument('--dataset', default='')
 
parser.add_argument('--max_len', default=140, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--num_classes', default=2, type=int)
parser.add_argument('--batch', default=8, type=int)
 
# parser.add_argument('--dataset', default='Sarcasm')
# parser.add_argument('--max_len', default=60, type=int)
# parser.add_argument('--epochs', default=6, type=int)
# parser.add_argument('--num_classes', default=2, type=int)
# parser.add_argument('--batch', default=64, type=int)
 
parser.add_argument('--ntransformer', default='Albert')
 
parser.add_argument('--device', default='cuda')
parser.add_argument('--weight', action='store_false')
parser.add_argument('--lr', default=2e-8, type=float)
parser.add_argument('--grad_clip', default=5., type=float)
parser.add_argument('--seed', default=1, type=int)
parser.add_argument('--notes', default=None)
parser.add_argument('--tensorboard', action='store_false')
args = parser.parse_known_args()
print(args)
 
# %%
MAX_LEN = args[0].max_len
N_EPOCHS = args[0].epochs
NUM_CLASSES = args[0].num_classes
BATCH_SIZE = args[0].batch
EPSILON = 10e-13
LR = args[0].lr
# DATASET = 'Dataset/' + args[0].dataset
DATASET = '' + args[0].dataset
WEIGHT = args[0].weight
GRAD_CLIP = args[0].grad_clip
device = torch.device(args[0].device) if torch.cuda.is_available() else torch.device('cpu')
TRANSFORMER = args[0].ntransformer
notes = args[0].notes
 
SEED = args[0].seed
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
 
hyper_parameters = {'batch': BATCH_SIZE, 'lr': LR, 'weight': WEIGHT, 'transformer': TRANSFORMER, 'seed': SEED}
 
# %%
if TRANSFORMER == 'Albert':
    # tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
    # transformer = AutoModelForMaskedLM.from_pretrained("nghuyong/ernie-2.0-en")
 
    tokenizer = transformers.AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
    transformer = transformers.AlbertModel.from_pretrained("albert-base-v2")
    
    # tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-tiny")
    # transformer = AutoModelForMaskedLM.from_pretrained("nghuyong/ernie-tiny")
 
    # tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")
    # transformer = AutoModelForMaskedLM.from_pretrained("nghuyong/ernie-2.0-large-en")
 
    #   erine_configuration = AutoConfig()
#  # Initializing an ALBERT-base style configuration
#   erine_base_configuration = AutoConfig(
#       hidden_size=768,
#       num_attention_heads=12,
#       intermediate_size=3072,
#   )
 
#  # Initializing a model from the ALBERT-base style configuration
#   model = AlbertModel(albert_xxlarge_configuration)
 
# # Accessing the model configuration
#   configuration = model.config
 
 
 
elif TRANSFORMER == 'DistilBert':
    tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
    transformer = transformers.DistilBertModel.from_pretrained("distilbert-base-uncased")
 
# %%
 
if args[0].tensorboard:
    time_string = (datetime.datetime.utcnow() + datetime.timedelta(seconds=12600)).replace(microsecond=0).isoformat('_')
    hyper_parameters_string = '--'.join(
        [k + '=' + str(v) for k, v in hyper_parameters.items()])
    logdir = f'single_runs/{args[0].dataset}/{hyper_parameters_string}'
    # if notes is not None:
    #     logdir += notes
    if os.path.exists(logdir):
        # rmtree(logdir)
        logdir += '/' + time_string
    tb = tensorboard.SummaryWriter(log_dir=logdir)
 
 
# %%
 
def df2dl(df,MAX_LEN_N):
    df = df.dropna()
    df.text = df.text.apply(lambda x: "[CLS] " + x + ' [SEP]')
    tokenized_texts = df.text.apply (lambda x: tokenizer.tok(' '.join(x.split()[:MAX_LEN_N])))
    token_ids = tokenized_texts.apply(tokenizer.convert_tokens_to_ids)
    token_ids_matrix = np.array(
        token_ids.apply(lambda x: x[:MAX_LEN_N] + [0] * max(0, MAX_LEN_N - len(x))).tolist()).astype('int64')
    attention_matrix = (token_ids_matrix != 0).astype('float')
    dataset = TensorDataset(torch.tensor(token_ids_matrix), torch.tensor(attention_matrix),
                            torch.tensor(np.array(df.label)))
    data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return data_loader
 
 
df = pd.read_csv('HateEval/train.csv')
df.dropna(inplace=True)
label_counts = df.label.value_counts(normalize=True)
train_data_loader = df2dl(df,78)
df = pd.read_csv('HateEval/test.csv')
test_data_loader = df2dl(df,140)
 
 
# %%
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer = transformer
        self.ff = nn.Sequential(
            nn.Linear(768, 100),
            nn.ReLU(),
            nn.Linear(100, NUM_CLASSES if NUM_CLASSES > 2 else 1)
        )
 
    def forward(self, batch_input, batch_mask):
      print("start", batch_input.shape)
      xx = self.transformer(batch_input, batch_mask)
      # print("part1 \n")
      # print(xx[0][:, 0, :].shape)
      xx = self.ff(xx[0][:, 0, :]).squeeze()
      print("part2", xx.shape)
      return xx
 
 
model = MyModel()
model = model.to(device)
# %%
param_optimizer = list(model.named_parameters())
 
# no_decay = ['bias', 'gamma', 'beta']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.0}
# ]
 
 
optimizer = AdamW(model.parameters(), lr=LR)
 
 
# %%
def calc_confusion_matrix(y, y_hat):
    with torch.no_grad():
        if NUM_CLASSES > 2:
            pred = y_hat.argmax(-1)
        else:
            pred = y_hat.gt(0)
 
    m = confusion_matrix(y.cpu(), pred.cpu(), range(NUM_CLASSES))
    return m
 
 
def calc_metrics(m):
    p = (m.diagonal() / m.sum(0).clip(EPSILON)).mean()
    r = (m.diagonal() / m.sum(1).clip(EPSILON)).mean()
    f1 = ((2 * p * r) / (p + r)).mean()
    accu = m.diagonal().sum() / m.sum()
    return p, r, f1, accu
 
 
train_weights = (1 / label_counts)
train_weights = train_weights.to_numpy() / train_weights.sum()
train_weights = torch.Tensor(train_weights).to(device)
 
if WEIGHT:
    if NUM_CLASSES > 2:
        criterion = nn.CrossEntropyLoss(reduction='sum', weight=train_weights)
    else:
        criterion = nn.BCEWithLogitsLoss(reduction='sum', pos_weight=train_weights[1])
else:
    if NUM_CLASSES > 2:
        criterion = nn.CrossEntropyLoss(reduction='sum')
    else:
        criterion = nn.BCEWithLogitsLoss(reduction='sum')
 
metrics_all = []
f_one=[]
accuracy=[]
flag = True
for i_epoch in range(1, N_EPOCHS + 1):
 
    model.train()
 
    # Tracking variables
    train_loss = 0
    train_confusion_table = np.zeros((NUM_CLASSES, NUM_CLASSES))
    train_total = 0
    metrics_train = ()
 
    p_bar = tqdm(train_data_loader)
    for step, batch in enumerate(p_bar):
        batch = tuple(t.to(device) for t in batch)
        batch_input, batch_mask, batch_y = batch
        optimizer.zero_grad()
 
        # print("for")
        # print(batch_input.shape)
        # print(batch_mask.shape)
        # break
        logits = model.forward(batch_input, batch_mask)
        # break
        
        loss = criterion(logits, batch_y if NUM_CLASSES > 2 else batch_y.float())
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        assert grad_norm >= 0, 'encounter nan in gradients.'
        optimizer.step()
 
        train_confusion_table += calc_confusion_matrix(batch_y, logits)
        train_total += batch_input.size(0)
        train_loss += loss.item()
 
        metrics_train = (
            train_loss / train_total,
            *calc_metrics(train_confusion_table),
        )
 
        p_bar.set_description(
 
            '[ EP {0:02d} ]'
            '[ TRN LS: {1:.4f} PR: {2:.4f} F1: {4:.4f} AC: {5:.4f}]'
                .format(i_epoch, *metrics_train))
 
    if args[0].tensorboard:
        for k, v in zip(['train_loss', 'train_precision', 'train_recall', 'train_f1', 'train_accuracy'], metrics_train):
            tb.add_scalar(k, v * 100, global_step=i_epoch)
 
    with torch.no_grad():
        # Set our model to testing mode (as opposed to evaluation mode)
        model.eval()
 
        # Tracking variables
        # test_loss = 0
        test_confusion_table = np.zeros((NUM_CLASSES, NUM_CLASSES))
        test_total = 0
 
        # test the data for one epoch
        p_bar = tqdm(test_data_loader)
        for step, batch in enumerate(p_bar):
            batch = tuple(t.to(device) for t in batch)
            batch_input, batch_mask, batch_y = batch
 
            logits = model(batch_input, batch_mask)
 
            test_confusion_table += calc_confusion_matrix(batch_y, logits)
            test_total += batch_input.size(0)
            # test_loss += loss.item()
 
            metrics_test = (
                # test_loss / test_total,
                *calc_metrics(test_confusion_table),
            )
            
            p_bar.set_description(
                '[ EP {0:02d} ]'
                '[ TST F1: {3:.4f} AC: {4:.4f}]'
                    .format(i_epoch, *metrics_test))
            
    f_one.append(metrics_test[2])
    accuracy.append(metrics_test[3])
    # print(len(patience) , patience[-1])
    if(len(f_one)>20):
      if(f_one[-1]<f_one[-21]):
        index = f_one.index(max(f_one))
        print("\nearly stopping at {} , max F1 is {:.4f} , max accuracy is {:.4f}\
         , in epoch {}".format(i_epoch , max(f_one) , accuracy[index], index+1))
        
        a = "\nearly stopping at {} , max F1 is {:.4f} , max accuracy is  {:.4f} ,\
         in epoch {}".format(i_epoch , max(f_one) , accuracy[index], index+1)
 
        with open("hateResult.txt", 'w') as writer:
          writer.write(a)
        flag = False
        break           
 
    if args[0].tensorboard:
        for k, v in zip(['test_precision', 'test_recall', 'test_f1', 'test_accuracy'], metrics_test):
            tb.add_scalar(k, v * 100, global_step=i_epoch)
 
    metrics_all.append((*metrics_train, *metrics_test))
 
# metrics = np.array(metrics_all).T
# best_index = metrics[-2].argmax()
# best_metrics = metrics.T[best_index]
 
# if args[0].tensorboard:
#     a = (hyper_parameters,
#          dict(zip(['z_precision', 'z_recall', 'z_f1', 'z_accuracy', 'z_ep'], best_metrics[-4:])))
 
#     for j in hparams(*a):
#         tb.file_writer.add_summary(j)
 
#     for k, v in a[1].items():
#         tb.add_scalar(k, v)
 
#     tb.close()
 
# %%

(Namespace(batch=8, dataset='', device='cuda', epochs=100, grad_clip=5.0, lr=2e-08, max_len=140, notes=None, ntransformer='Albert', num_classes=2, seed=1, tensorboard=True, weight=True), ['-f', '/root/.local/share/jupyter/runtime/kernel-e3fb01a7-5af2-4d1c-9308-e812dd3219d6.json'])


AttributeError: ignored

In [ ]:
# !python main.py --dataset HateEval --num_classes 2 --epochs 10

In [ ]:
# !python main_mt.py

In [ ]:
import os
import random
import datetime, time
from shutil import rmtree

from torch.utils import tensorboard
import torch
from sklearn.metrics import confusion_matrix
from torch import nn
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard.summary import hparams
import transformers
import tqdm
from torch.nn import functional as F
from tqdm import tqdm
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

# %%
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--ntransformer', default='Albert')

parser.add_argument('--device', default='cuda')
parser.add_argument('--weight', action='store_false')
parser.add_argument('--lr', default=2e-8, type=float)
parser.add_argument('--grad_clip', default=5., type=float)
parser.add_argument('--seed', default=1, type=int)
parser.add_argument('--notes', default=None)
args = parser.parse_known_args()
print(args)

# %%
# MAX_LEN_H = 140
# MAX_LEN_O = 78

MAX_LEN = 60
N_EPOCHS = 200
# N_EPOCHS = 1

BATCH_SIZE = 24

EPSILON = 10e-13
LR = args[0].lr
WEIGHT = args[0].weight
# print('moh',WEIGHT)
GRAD_CLIP = args[0].grad_clip
device = torch.device(args[0].device) if torch.cuda.is_available() else torch.device('cpu')
TRANSFORMER = args[0].ntransformer
notes = args[0].notes

SEED = args[0].seed
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

hyper_parameters = {'batch': BATCH_SIZE, 'lr': LR, 'weight': WEIGHT, 'transformer': TRANSFORMER, 'seed': SEED}

# %%
if TRANSFORMER == 'Albert':
    tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
    transformer = AutoModelForMaskedLM.from_pretrained("nghuyong/ernie-2.0-en")


    #tokenizer = transformers.AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
    #transformer = transformers.AlbertModel.from_pretrained("albert-base-v2")




elif TRANSFORMER == 'DistilBert':
    tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
    transformer = transformers.DistilBertModel.from_pretrained("distilbert-base-uncased")

# %%

# time_string = (datetime.datetime.utcnow() + datetime.timedelta(seconds=12600)).replace(microsecond=0).isoformat('_')
# hyper_parameters_string = '--'.join(
#     [k + '=' + str(v) for k, v in hyper_parameters.items()])
# logdir = f'single_runs/{args[0].dataset}/{hyper_parameters_string}'
# # if notes is not None:
# #     logdir += notes
# if os.path.exists(logdir):
#     # rmtree(logdir)
#     logdir += '/' + time_string
# tb = tensorboard.SummaryWriter(log_dir=logdir)

#%%



# %%

# class CustomBCELoss(nn.Module):
#     def __init__(self):
#       self.sigmas =


def df2dl(df,max_len, batch_size, shuffle):
    # if hate==True:
    #   max_len = 140
    # else:
    #   max_len = 78
    df = df.dropna()
    df.text = df.text.apply(lambda x: "[CLS] " + x + ' [SEP]')
    tokenized_texts = df.text.apply(lambda x: tokenizer.tokenize(' '.join(x.split()[:max_len])))
    token_ids = tokenized_texts.apply(tokenizer.convert_tokens_to_ids)
    token_ids_matrix = np.array(
        token_ids.apply(lambda x: x[:max_len] + [0] * max(0, max_len - len(x))).tolist()).astype(
        'int64')
    attention_matrix = (token_ids_matrix != 0).astype('float')
    dataset = TensorDataset(torch.tensor(token_ids_matrix), torch.tensor(attention_matrix),
                            torch.tensor(np.array(df.label)))
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader


dataset_names = 'OffenseEval HateEval Sentiment'.split(' ')
num_datasets = len(dataset_names)
# datasets = [dict() for _ in range(len(dataset_names))]

train_data_loaders = []
test_data_loaders = []
criterions = []
dl_num_classes = []

for d_index, d in enumerate(dataset_names):
    df = pd.read_csv(f'{d}/train.csv')
    df.dropna(inplace=True)

    label_counts = df.label.value_counts(normalize=True)
    train_weights = (1 / label_counts)
    train_weights = train_weights.to_numpy() / train_weights.sum()
    train_weights = torch.Tensor(train_weights).to(device)
    # datasets[d_index]['tw'] = train_weights

    if d == "OffenseEval":
      max_len = 78
    elif d == "HateEval":
      max_len = 140
    else:
      max_len = 120
      
    train_data_loader = df2dl(df, max_len, BATCH_SIZE, True)

    train_data_loaders.append(train_data_loader)


    df = pd.read_csv(f'{d}/test.csv')

    # if d == "OffenseEval":
    #   test_data_loader = df2dl(df, 78, BATCH_SIZE, False)
    # else:
    #   test_data_loader = df2dl(df, 120, BATCH_SIZE, False)

    test_data_loader = df2dl(df, 120, BATCH_SIZE, False)

    test_data_loaders.append(test_data_loader)

    num_classes = {
      'Sentiment':2,
      'OffenseEval': 2, 
      'HateEval': 2}[d]
    dl_num_classes.append(num_classes)

    if WEIGHT:
        # print(WEIGHT)
        # print('where you are')
        if num_classes > 2:
            criterion = nn.CrossEntropyLoss(reduction='sum', weight=train_weights)
        else:
            criterion = nn.BCEWithLogitsLoss(reduction='sum', pos_weight=train_weights[1])
    else:
        # print('where i am')
        if num_classes > 2:
            criterion = nn.CrossEntropyLoss(reduction='sum')
        else:
            criterion = nn.BCEWithLogitsLoss(reduction='sum')

    criterions.append(criterion)

#%%
#کاری که میکنه اینکه 
#اول میشماره چنتا بچ داخل هر دیتالودر هستش
#
def shuffle_data_loaders(dataloaders):

    batch_counts = [len(dl) for dl in dataloaders]
    dataloader_iters = [iter(dl) for dl in dataloaders]

    dataloader_indices = []
    for idx, count in enumerate(batch_counts):
        dataloader_indices.extend([idx] * count)

#injaro bayad dune dune konim
    random.shuffle(dataloader_indices)

    for index in dataloader_indices:
        yield next(dataloader_iters[index]), index, criterions[index], dl_num_classes[index]


# %%
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer = transformer
        self.ffs = nn.ModuleList([nn.Sequential(
            nn.Linear(30522, 1000),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(1000, 100),
            nn.ReLU(),
            nn.Linear(100, nc if nc > 2 else 1)
        ) for nc in dl_num_classes])

    def forward(self, batch_input, batch_mask, task_id):
        xx = self.transformer(batch_input, batch_mask)
        xx = self.ffs[task_id](xx[0][:, 0, :]).squeeze()
        return xx


model = MyModel()
model = model.to(device)
# %%
# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'gamma', 'beta']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.0}
# ]


optimizer = AdamW(model.parameters(), lr=LR)


# %%
def calc_confusion_matrix(y, y_hat, num_classes):
    with torch.no_grad():
        if num_classes > 2:
            pred = y_hat.argmax(-1)
        else:
            pred = y_hat.gt(0)

    m = confusion_matrix(y.cpu(), pred.cpu(), range(num_classes))
    return m


def calc_metrics(m):
    p = (m.diagonal() / m.sum(0).clip(EPSILON)).mean()
    r = (m.diagonal() / m.sum(1).clip(EPSILON)).mean()
    f1 = ((2 * p * r) / (p + r)).mean()
    accu = m.diagonal().sum() / m.sum()
    return p, r, f1, accu






metrics_all = []
hate_f_one=[]
hate_accuracy=[]

sent_f_one=[]
sent_accuracy=[]

offe_f_one=[]
offe_accuracy=[]


# class MultiLossLayer():
#   def __init__(self, loss_list):
#     self._loss_list = loss_list
#     self._sigmas_sq = []
#     for i in range(len(self._loss_list)):
#       self._sigmas_sq.append(slim.variable('Sigma_sq_' + str(i), dtype=tf.float32, shape=[], initializer=tf.initializers.random_uniform(minval=0.2, maxval=1)))

#   def get_loss(self):
#     factor = tf.div(1.0, tf.multiply(2.0, self._sigmas_sq[0]))
#     loss = tf.add(tf.multiply(factor, self._loss_list[0]), tf.log(self._sigmas_sq[0]))
#     for i in range(1, len(self._sigmas_sq)):
#       factor = tf.div(1.0, tf.multiply(2.0, self._sigmas_sq[i]))
#       loss = tf.add(loss, tf.add(tf.multiply(factor, self._loss_list[i]), tf.log(self._sigmas_sq[i])))
#     return loss




train_desc_string = '[ EP {0:02d} TRN ]' +
                    ''.join([f"[ {dataset_names[i][:4]} LS: {{{1 + 5 * i}:.3f}} \
                    F1: {{{4 + 5 * i}:.3f}} AC: {{{5 + 5 * i }:.3f}} ]" for i in range(num_datasets)])
flag=True

# sigma_one = Variable(Uniform(torch.tensor([0.2]), torch.tensor([1.0])))


# sigma_two = Variable(Uniform(torch.tensor([0.2]), torch.tensor([1.0])))
  

for i_epoch in range(1, N_EPOCHS + 1):

    model.train()

    # for d_index, dataset in enumerate(datasets):
    #     print(dataset_names[d_index])
    # train_data_loader = dataset['train']
    # criterion = dataset['criterion']
    # num_classes = dataset['num_classes']
    train_loss = [0] * num_datasets
    train_confusion_table = [np.zeros((n, n)) for n in dl_num_classes]
    train_total = [0] * num_datasets

    p_bar = tqdm(shuffle_data_loaders(train_data_loaders), total=sum(len(dl) for dl in train_data_loaders))
    # print('--'*2,p_bar)

    metrics_train = np.zeros((5 * num_datasets,))

    for step, stuff in enumerate(p_bar):
        batch, d_index, criterion, num_classes = stuff
        batch = tuple(t.to(device) for t in batch)
        batch_input, batch_mask, batch_y = batch
        optimizer.zero_grad()

        logits = model(batch_input, batch_mask, d_index)
        labels = batch_y if num_classes > 2 else batch_y.float()

        # print('*'*5, labels)
        # for i in range(1, len(self._sigmas_sq)):
        #   factor = tf.div(1.0, tf.multiply(2.0, self._sigmas_sq[i]))
        #   loss = tf.add(loss, tf.add(tf.multiply(factor, self._loss_list[i]), tf.log(self._sigmas_sq[i])))
        # part_one = (1/(2*sigma_one))

        loss = criterion(logits, batch_y if num_classes > 2 else batch_y.float())
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        assert grad_norm >= 0, 'encounter nan in gradients.'
        optimizer.step()

        train_confusion_table[d_index] += calc_confusion_matrix(batch_y, logits, num_classes)
        train_total[d_index] += batch_input.size(0)
        train_loss[d_index] += loss.item()

        metrics_train[d_index * 5: (d_index + 1) * 5] = np.array([
            train_loss[d_index] / train_total[d_index], *calc_metrics(train_confusion_table[d_index]),
        ])

        p_bar.set_description(train_desc_string.format(i_epoch, *metrics_train))


    with torch.no_grad():
        # Set our model to testing mode (as opposed to evaluation mode)
        model.eval()

        for d_index, (test_data_loader, num_classes) in enumerate(zip(test_data_loaders, dl_num_classes)):

            # Tracking variables
            test_confusion_table = np.zeros((num_classes, num_classes))
            test_total = 0
            # test the data for one epoch
            p_bar = tqdm(test_data_loader)
            for step, batch in enumerate(p_bar):
                batch = tuple(t.to(device) for t in batch)
                batch_input, batch_mask, batch_y = batch

                logits = model(batch_input, batch_mask, d_index)

                test_confusion_table += calc_confusion_matrix(batch_y, logits, num_classes)
                test_total += batch_input.size(0)
                # test_loss += loss.item()

                metrics_test = (
                    # test_loss / test_total,
                    *calc_metrics(test_confusion_table),
                )

                p_bar.set_description(
                    '[ EP {0:02d} TST ]'
                    '[ {1} F1: {4:.4f} AC: {5:.4f}]'
                        .format(i_epoch, dataset_names[d_index][:4], *metrics_test))
                

            if (dataset_names[d_index][:4]=='Offe'):
              offe_f_one.append(metrics_test[2])
              offe_accuracy.append(metrics_test[3])
            elif (dataset_names[d_index][:4]=='Hate'):
              hate_f_one.append(metrics_test[2])
              hate_accuracy.append(metrics_test[3])
            else:
              sent_f_one.append(metrics_test[2])
              sent_accuracy.append(metrics_test[3])

            if (len(hate_f_one)>20 and len(offe_f_one)>20 and len(sent_f_one)>20):

              if( hate_f_one[-1]<hate_f_one[-21] and 
                 offe_f_one[-1]<offe_f_one[-21] and
                 sent_f_one[-1]<sent_f_one[-21]):
                
                offe_index = offe_f_one.index(max(offe_f_one))
                hate_index = hate_f_one.index(max(hate_f_one))
                sent_index = sent_f_one.index(max(sent_f_one))

                print("\nearly stopping at {}\n \
                        Offensive max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Hate max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Sent max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}".format(i_epoch , 
                            max(offe_f_one) , offe_accuracy[offe_index], offe_index+1, 
                            max(hate_f_one) , hate_accuracy[hate_index], hate_index+1, 
                            max(sent_f_one) , sent_accuracy[sent_index], sent_index+1))
                
                a = ("\nearly stopping at {}\n \
                        Offensive max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Hate max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Sent max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}".format(i_epoch , 
                            max(offe_f_one) , offe_accuracy[offe_index], offe_index+1, 
                            max(hate_f_one) , hate_accuracy[hate_index], hate_index+1, 
                            max(sent_f_one) , sent_accuracy[sent_index], sent_index+1))
                

                with open("multiResult.txt", 'a+') as writer:
                    writer.write(a)
                flag = False
                break


    if(not(flag)):
      break




# %%

SyntaxError: ignored

In [ ]:
            if(dataset_names[d_index][:4]=='Offe'):
              offe_f_one.append(metrics_test[2])
              offe_accuracy.append(metrics_test[3])
            elif:
              hate_f_one.append(metrics_test[2])
              hate_accuracy.append(metrics_test[3])

            else:
              sent_f_one.append(metrics_test[2])
              sent_accuracy.append(metrics_test[3])

            if(len(hate_f_one)>20 and len(offe_f_one)>20 and len(sent_f_one)>20):

              if( hate_f_one[-1]<hate_f_one[-21] and 
                 offe_f_one[-1]<offe_f_one[-21] and
                 sent_f_one[-1]<sent_f_one[-21]):
                
                offe_index = offe_f_one.index(max(offe_f_one))
                hate_index = hate_f_one.index(max(hate_f_one))
                sent_index = sent_f_one.index(max(sent_f_one))

                print("\nearly stopping at {}\n \
                        Offensive max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Hate max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Sent max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}".format(i_epoch , 
                            max(offe_f_one) , offe_accuracy[offe_index], offe_index+1, 
                            max(hate_f_one) , hate_accuracy[hate_index], hate_index+1, 
                            max(sent_f_one) , sent_accuracy[sent_index], sent_index+1))
                
                a = ("\nearly stopping at {}\n \
                        Offensive max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Hate max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}\n \
                        Sent max F1 is {:.4f} , max accuracy is {:.4f} in epoch {}".format(i_epoch , 
                            max(offe_f_one) , offe_accuracy[offe_index], offe_index+1, 
                            max(hate_f_one) , hate_accuracy[hate_index], hate_index+1, 
                            max(sent_f_one) , sent_accuracy[sent_index], sent_index+1))


In [ ]:
import torch
import torch.nn as nn
loss = nn.CrossEntropyLoss()

In [ ]:

input = torch.tensor([ 0.0917,  0.0051,  0.1998,  0.0585,  0.0304,  0.0237,  0.0844,  0.2877,
        -0.1244,  0.1348,  0.0177,  0.0103,  0.1387,  0.0503,  0.1195,  0.0242,
         0.0085, -0.0014,  0.0990,  0.0315, -0.0265,  0.2276,  0.1223,  0.2129], requires_grad=True)
target = torch.tensor([1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
        1., 1., 0., 1., 1., 1.])


In [ ]:
output = loss(input, target)

IndexError: ignored

In [ ]:
a = [1, 2, 3]
a[:2]

[1, 2]